In [46]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
from address_compare.constants import DIRECTIONS, STREET_TYPES, UNIT_TYPES
from address_compare.parsers import hyphen_parse
import re

In [30]:
addresses = pd.read_excel("data/washington state address training data.xlsx")

In [77]:
np.random.seed(100)
td = pd.concat([addresses[pd.isnull(addresses['Unit Number'])].sample(100), addresses[~pd.isnull(addresses['Unit Number'])]])
td.to_csv('data/crftrain1.csv', index = False)

In [25]:
COMPASS_POINTS_DICT = {'NORTH': {'N','NORTH','N.'},
                       'SOUTH': {'S','SOUTH','S.'},
                       'EAST': {'E','EAST','E.'},
                       'WEST': {'W','WEST','W.'},
                       'NE': {'NE','NE.','N.E.','NORTHEAST'},
                       'NW': {'NW','NW.','N.W.','NORTHWEST'},
                       'SE': {'SE','SE.','S.E.','SOUTHEAST'},
                       'SW': {'SW','SW.','S.W.','SOUTHWEST'}}

In [27]:
new_dict = dict()
for k, values in COMPASS_POINTS_DICT.items():
    for v in values:
        new_dict[v] = k

In [95]:
mturk1 = pd.read_csv('data/mturk1.csv')

In [101]:
mturk1['tags'] = mturk1['plain'].apply(lambda x: ' '.join('[%s]' % t for t in hyphen_parse(x)))
mturk1['number'] = mturk1['plain'].apply(lambda x: len(hyphen_parse(x)))

In [100]:
mturk1['tags']

0                          [612] [S] [ALASKA] [ST]
1               [540] [RONLEE] [LN] [NW] [STE] [B]
2                       [624] [SUNSET] [PARK] [DR]
3              [1874] [BATUM] [RD] [N] [STE] [100]
4           [15026] [E] [BREVIER] [RD] [SUITE] [A]
5        [750] [E] [WAPATO] [LAKE] [RD] [UNIT] [C]
6                               [123] [ELWHA] [RD]
7                        [221] [OMAK] [RIVER] [RD]
8                [298] [POORMAN] [CREEK] [RD] [#A]
9       [2505] [N] [COMMERCIAL] [AVE] [UNIT] [C/D]
10                         [2200] [N] [RIVER] [RD]
11                   [1123] [351ST] [ST] [STE] [B]
12                           [2407] [TALLEY] [WAY]
13                 [1824] [RD] [N] [NE] [UNIT] [3]
14                            [1811] [BAKER] [WAY]
15              [3660] [LINDSAY] [RD] [SUITE] [1M]
16                [4055] [HAMMER] [DR] [STE] [103]
17                          [1788] [3RD] [AVE] [W]
18                  [8531] [POSTMA] [RD] [STE] [P]
19                           [3

In [5]:
available_labels = ["UNIT_TYPE", "UNIT_NUMBER", "STREET_NUMBER", "PRE_DIRECTION",
"STREET_NAME", "STREET_TYPE", "POST_DIRECTION", "UNKNOWN"]
 
results1 = pd.read_csv('C:/Users/colin/Downloads/Batch_3033837_batch_results (1).csv')
results1.head()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.ID,Input.plain,Input.tags,Input.number,Answer.WritingTexts,Approve,Reject
0,3EPG8DX9LKQCFDYSKHJISD4TM765PU,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,0% (0/0),0% (0/0),0% (0/0),356,612 S ALASKA ST,[612] [S] [ALASKA] [ST],4,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,NaN,NaN
1,3EPG8DX9LKQCFDYSKHJISD4TM765PU,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,100% (3/3),100% (3/3),100% (3/3),356,612 S ALASKA ST,[612] [S] [ALASKA] [ST],4,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,NaN,NaN
2,3EPG8DX9LKQCFDYSKHJISD4TM765PU,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,0% (0/0),0% (0/0),0% (0/0),356,612 S ALASKA ST,[612] [S] [ALASKA] [ST],4,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,NaN,NaN
3,3OPLMF3EU5NQEHP0WPMX5DVV8I4LN1,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,100% (3/3),100% (3/3),100% (3/3),821,540 RONLEE LN NW STE B,[540] [RONLEE] [LN] [NW] [STE] [B],6,STREET_NUMBER STREET_NAME STREET_TYPE POST_DI...,NaN,NaN
4,3OPLMF3EU5NQEHP0WPMX5DVV8I4LN1,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,0% (0/0),0% (0/0),0% (0/0),821,540 RONLEE LN NW STE B,[540] [RONLEE] [LN] [NW] [STE] [B],6,STREET_NUMBER STREET_NAME STREET_NAME POST_DIR...,NaN,NaN


In [20]:
available_labels = ["UNIT_TYPE", "UNIT_NUMBER", "STREET_NUMBER", "PRE_DIRECTION",
"STREET_NAME", "STREET_TYPE", "POST_DIRECTION", "UNKNOWN"]
valid_tags = results1['Answer.WritingTexts'].apply(lambda x: all([k in available_labels for k in x.split()]))
right_number_of_tags = results1['Answer.WritingTexts'].apply(lambda x: len(x.split())) == results1['Input.number']
results1['Approve'] = np.where(valid_tags & right_number_of_tags, 'x', '')
results1['Reject'] = np.where(results1['Approve'] == '', 'x', '')

In [24]:
results1.to_csv('data/TrainingApproval.csv')

_wait for updated results from the rejected ones_

In [50]:
results2 = pd.read_csv('C:/Users/colin/Downloads/Batch_3033837_batch_results (4).csv')
results2.head()
valid_tags2 = results2['Answer.WritingTexts'].apply(lambda x: all([k in available_labels for k in x.split()]))
right_number_of_tags2 = results2['Answer.WritingTexts'].apply(lambda x: len(x.split())) == results2['Input.number']
results2['Approve'] = np.where(valid_tags2 & right_number_of_tags2, 'x', '')
results2['Reject'] = np.where(results2['Approve'] == '', 'x', '')
results2.to_csv('data/TrainingApproval2.csv')

In [59]:
df = results2[results2['Approve']=='x']
import re
df['formatted_response'] = df['Answer.WritingTexts'].apply(lambda x: re.sub(' +', ' ', x))
df

C:\Users\colin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.ID,Input.plain,Input.tags,Input.number,Answer.WritingTexts,Approve,Reject,formatted_response
0,3EPG8DX9LKQCFDYSKHJISD4TM765PU,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,90% (38/42),90% (38/42),356,612 S ALASKA ST,[612] [S] [ALASKA] [ST],4,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,x,,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...
1,3EPG8DX9LKQCFDYSKHJISD4TM765PU,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,98% (165/169),98% (165/169),356,612 S ALASKA ST,[612] [S] [ALASKA] [ST],4,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,x,,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...
2,3EPG8DX9LKQCFDYSKHJISD4TM765PU,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,98% (109/111),98% (109/111),356,612 S ALASKA ST,[612] [S] [ALASKA] [ST],4,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,x,,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...
3,3OPLMF3EU5NQEHP0WPMX5DVV8I4LN1,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,98% (165/169),98% (165/169),821,540 RONLEE LN NW STE B,[540] [RONLEE] [LN] [NW] [STE] [B],6,STREET_NUMBER STREET_NAME STREET_TYPE POST_DI...,x,,STREET_NUMBER STREET_NAME STREET_TYPE POST_DIR...
4,3OPLMF3EU5NQEHP0WPMX5DVV8I4LN1,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,100% (15/15),100% (15/15),821,540 RONLEE LN NW STE B,[540] [RONLEE] [LN] [NW] [STE] [B],6,STREET_NUMBER STREET_NAME STREET_NAME POST_DIR...,x,,STREET_NUMBER STREET_NAME STREET_NAME POST_DIR...
5,3OPLMF3EU5NQEHP0WPMX5DVV8I4LN1,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,98% (109/111),98% (109/111),821,540 RONLEE LN NW STE B,[540] [RONLEE] [LN] [NW] [STE] [B],6,STREET_NUMBER STREET_NAME STREET_TYPE POST_DIR...,x,,STREET_NUMBER STREET_NAME STREET_TYPE POST_DIR...
6,306W7JMRYYYTY1GKZA8K9D9V7TS8BE,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,100% (2/2),100% (2/2),368,624 SUNSET PARK DR,[624] [SUNSET] [PARK] [DR],4,UNIT_NUMBER STREET_NAME STREET_TYPE POST_DIREC...,x,,UNIT_NUMBER STREET_NAME STREET_TYPE POST_DIREC...
7,306W7JMRYYYTY1GKZA8K9D9V7TS8BE,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,98% (165/169),98% (165/169),368,624 SUNSET PARK DR,[624] [SUNSET] [PARK] [DR],4,STREET_NUMBER STREET_NAME STREET_NAME S

In [61]:
df['ValueCounts'] = df.groupby(['HITId', 'formatted_response'])['formatted_response'].transform('count')
df

C:\Users\colin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Last7DaysApprovalRate,Input.ID,Input.plain,Input.tags,Input.number,Answer.WritingTexts,Approve,Reject,formatted_response,ValueCounts
0,3EPG8DX9LKQCFDYSKHJISD4TM765PU,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,90% (38/42),356,612 S ALASKA ST,[612] [S] [ALASKA] [ST],4,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,x,,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,3
1,3EPG8DX9LKQCFDYSKHJISD4TM765PU,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,98% (165/169),356,612 S ALASKA ST,[612] [S] [ALASKA] [ST],4,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,x,,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,3
2,3EPG8DX9LKQCFDYSKHJISD4TM765PU,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,98% (109/111),356,612 S ALASKA ST,[612] [S] [ALASKA] [ST],4,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,x,,STREET_NUMBER PRE_DIRECTION STREET_NAME STREET...,3
3,3OPLMF3EU5NQEHP0WPMX5DVV8I4LN1,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,98% (165/169),821,540 RONLEE LN NW STE B,[540] [RONLEE] [LN] [NW] [STE] [B],6,STREET_NUMBER STREET_NAME STREET_TYPE POST_DI...,x,,STREET_NUMBER STREET_NAME STREET_TYPE POST_DIR...,2
4,3OPLMF3EU5NQEHP0WPMX5DVV8I4LN1,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,100% (15/15),821,540 RONLEE LN NW STE B,[540] [RONLEE] [LN] [NW] [STE] [B],6,STREET_NUMBER STREET_NAME STREET_NAME POST_DIR...,x,,STREET_NUMBER STREET_NAME STREET_NAME POST_DIR...,1
5,3OPLMF3EU5NQEHP0WPMX5DVV8I4LN1,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,98% (109/111),821,540 RONLEE LN NW STE B,[540] [RONLEE] [LN] [NW] [STE] [B],6,STREET_NUMBER STREET_NAME STREET_TYPE POST_DIR...,x,,STREET_NUMBER STREET_NAME STREET_TYPE POST_DIR...,2
6,306W7JMRYYYTY1GKZA8K9D9V7TS8BE,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,100% (2/2),368,624 SUNSET PARK DR,[624] [SUNSET] [PARK] [DR],4,UNIT_NUMBER STREET_NAME STREET_TYPE POST_DIREC...,x,,UNIT_NUMBER STREET_NAME STREET_TYPE POST_DIREC...,1
7,306W7JMRYYYTY1GKZA8K9D9V7TS8BE,3MCFX9UVS0L6IH88XBXSIDFVZSP86D,Tag the parts of an address,"Given an address, indicate which parts are str...","POS-tagging, reading, writing, tagging, natura...",$0.08,Sat Dec 02 23:57:44 PST 2017,3,BatchId:3033837;OriginalHitTemplateId:712361150;,300,...,98% (165/169),368,624 SUNSET PARK DR,[624] [SUNSET] [PARK] [DR],4,STREET_NUMBER STREET_NAME STREET_NAME STREET_...,x,,STREET_NUMBER STREET_NAME STREET_NAME STREET_TYPE,2
8,306W7JMRYYYTY1GKZA8K9D9V7TS8BE,3MCFX

In [73]:
import json
tagdf = df[df['ValueCounts'] > 1][['Input.plain', 'formatted_response']].drop_duplicates()

In [78]:
training_data = list()
for i, ad, r in tagdf.itertuples():
    training_data.append({'raw_address': ad, 'tags': r.split()})

print(json.dumps(training_data, indent = 2))

[
  {
    "raw_address": "612 S ALASKA ST",
    "tags": [
      "STREET_NUMBER",
      "PRE_DIRECTION",
      "STREET_NAME",
      "STREET_TYPE"
    ]
  },
  {
    "raw_address": "540 RONLEE LN NW STE B",
    "tags": [
      "STREET_NUMBER",
      "STREET_NAME",
      "STREET_TYPE",
      "POST_DIRECTION",
      "UNIT_TYPE",
      "UNIT_NUMBER"
    ]
  },
  {
    "raw_address": "624 SUNSET PARK DR",
    "tags": [
      "STREET_NUMBER",
      "STREET_NAME",
      "STREET_NAME",
      "STREET_TYPE"
    ]
  },
  {
    "raw_address": "1874 BATUM RD N  STE 100",
    "tags": [
      "STREET_NUMBER",
      "STREET_NAME",
      "STREET_TYPE",
      "POST_DIRECTION",
      "UNIT_TYPE",
      "UNIT_NUMBER"
    ]
  },
  {
    "raw_address": "15026 E BREVIER RD SUITE A",
    "tags": [
      "STREET_NUMBER",
      "PRE_DIRECTION",
      "STREET_NAME",
      "STREET_TYPE",
      "UNIT_TYPE",
      "UNIT_NUMBER"
    ]
  },
  {
    "raw_address": "750 E WAPATO LAKE RD UNIT C",
    "tags": [
      "STR

In [81]:
with open('data/tagged_addresses.json', 'w') as f:
    json.dump(training_data, f)